In [2]:
import numpy as np
import open3d
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.neighbors import NearestNeighbors
import datetime


import random

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Hyper paramters
- epsilon
- minpts
- voxelsize
- Min_pts in cluster for bounding box
- No. of clusters


In [2]:
pcd = open3d.io.read_point_cloud("assignment/pcd_10.pcd", format = 'auto')
open3d.visualization.draw_geometries([pcd])

In [27]:
###Downsampling
downsample_pcd = pcd.voxel_down_sample(voxel_size = 0.2)
open3d.visualization.draw_geometries([downsample_pcd])
print(len(pcd.points))
print(len(downsample_pcd.points))

45785
24922


In [28]:
###Segmentation
_, inliers = downsample_pcd.segment_plane(distance_threshold = 0.3, ransac_n = 3, num_iterations=250)
inlier_cloud = downsample_pcd.select_by_index(inliers)
outlier_cloud = downsample_pcd.select_by_index(inliers, invert=True)
inlier_cloud.paint_uniform_color([1,0,0])
outlier_cloud.paint_uniform_color([0,0,1])
open3d.visualization.draw_geometries([inlier_cloud,outlier_cloud])

In [5]:
labels = np.array(outlier_cloud.cluster_dbscan(eps=0.55, min_points=10, print_progress=True))
max_label = labels.max()
colors = plt.get_cmap("tab20")(labels/(max_label if max_label >0 else 1))

colors[labels<0] = 0
outlier_cloud.colors = open3d.utility.Vector3dVector(colors[:,:3])
open3d.visualization.draw_geometries([outlier_cloud])

In [6]:
print(labels.shape)

(12315,)


In [7]:
Max_points=1000
Min_points=20
clusters = []
obbs=[]
indexes = pd.Series(range(len(labels))).groupby(labels,sort = False).apply(list).tolist()
for i in range(0,len(indexes)):
    nb_pts = len(outlier_cloud.select_by_index(indexes[i]).points)
    if nb_pts>Min_points:
        
        sub_cloud = outlier_cloud.select_by_index(indexes[i])
        obb = sub_cloud.get_axis_aligned_bounding_box()
        obb.color = (0,0,1)
        obbs.append(obb)



In [8]:
list_of_visuals = []
list_of_visuals.append(outlier_cloud)
list_of_visuals.extend(obbs)
list_of_visuals.append(inlier_cloud)
open3d.visualization.draw_geometries(list_of_visuals)

In [29]:
data = np.asarray([outlier_cloud.points])
data = data.reshape((len(outlier_cloud.points),3))


In [30]:
def euclidean_distance(p1,p2):
    return np.sqrt(np.sum((p1-p2)**2))

def get_neighbors(X, pt, epsilon, nn):
    indices = nn.radius_neighbors([X[pt]], radius=epsilon, return_distance=False)
    return indices[0]


def expand(X, clusters, point, neighbors, currentPoint, eps, minPts, nn):
    clusters[point] = currentPoint
    
    i = 0
    while i < len(neighbors):
        nextPoint = neighbors[i]
        
        if clusters[nextPoint] == -1:
            clusters[nextPoint] = currentPoint
        elif clusters[nextPoint] == 0:
            clusters[nextPoint] = currentPoint
            nextNeighbors = get_neighbors(X, nextPoint, eps, nn)
            
            if len(nextNeighbors) >= minPts:
                neighbors = np.concatenate((neighbors, nextNeighbors))
        
        i += 1
    

def optimized_DBSCAN(X, clusters, eps, minPts):
    
    currentPoint = 0
    nn = NearestNeighbors(radius=eps)
    nn.fit(X)
    
    for i in range(0, X.shape[0]):
        if clusters[i] != 0:
            continue
    
        neighbors = get_neighbors(X, i, eps, nn)

        if len(neighbors) < minPts:
            clusters[i] = -1

        else:
            currentPoint += 1
            expand(X, clusters, i, neighbors, currentPoint, eps, minPts, nn)
    
    return clusters

In [3]:
start = datetime.datetime.now()
cluster = [0]*data.shape[0]
c=optimized_DBSCAN(data,cluster,eps=0.9, minPts=6)
end = datetime.datetime.now()


NameError: name 'data' is not defined

In [ ]:
print('Time to perform clustering:', end-start)

In [38]:
c = np.array(c)

max_label = c.max()
colors = plt.get_cmap("tab20")(c/(max_label if max_label >0 else 1))

colors[c<0] = 0
outlier_cloud.colors = open3d.utility.Vector3dVector(colors[:,:3])
open3d.visualization.draw_geometries([outlier_cloud])

In [39]:
Max_points=1000
Min_points=20
clusters = []
obbs=[]
indexes = pd.Series(range(len(c))).groupby(c,sort = False).apply(list).tolist()
for i in range(0,len(indexes)):
    nb_pts = len(outlier_cloud.select_by_index(indexes[i]).points)
    if nb_pts>Min_points:
        
        sub_cloud = outlier_cloud.select_by_index(indexes[i])
        obb = sub_cloud.get_axis_aligned_bounding_box()
        obb.color = (0,0,1)
        obbs.append(obb)


In [41]:
list_of_visuals = []
list_of_visuals.append(outlier_cloud)
list_of_visuals.extend(obbs)
list_of_visuals.append(inlier_cloud)
open3d.visualization.draw_geometries(list_of_visuals)

In [ ]:

def get_neighbors(X,pt,epsilon):
    neighbour=[]
    for index in range(X.shape[0]):
        if euclidean_distance(X[pt],X[index])<epsilon:
            neighbour.append(index)
    return neighbour


